In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Лабораторна робота №4
## ФІ91 Корешков Михайло (Варіант 6)
Метод Якобі

In [2]:
A = np.array([
    [7.00, 0.88, 0.93, 1.21],
    [0.88, 4.16, 1.30, 0.15],
    [0.93, 1.30, 6.44, 2.00],
    [1.21, 0.15, 2.00, 9.00]
])

In [3]:
# Сферича норма
def S_A(A):
    return (A*A).sum()

# Діагональна частина сферичної норми
def S_d(A):
    d = A.diagonal()
    return (d*d).sum()

Обертання: 
$$T_{ij}: \begin{cases}
    t_{kk} = 1, k\ne i, k\ne j\\
    t_{ii} = c\\
    t_{jj} = c\\
    t_{ij} = s\\
    t_{ji} = -s
\end{cases} \quad s^2 + c^2 = 1$$
Обертання впливає лише на пару рядків та стовпців.
- Для $B=AT_{ij}$ змінюються **стовпці**: $\;B_i = cA_i + sA_j, \; B_j = -sA_i + cA_j$
- Для $C=T'_{ij}B$ змінюються **рядки**: $\;C^i = cB^i + sB^j, \; C^j = -sB^i + cB^j$


Суть обертання - анулювати елемент $c_{ij}$ матриці $C = T'_{ij}AT_{ij}$. 
$$c_{ij} = c(-sa_{ii}+ca_{ij})+s(-sa_{ji}+ca_{jj}) = cs(a_{jj}-a_{ii})+(c^2-s^2)a_{ij} = 0$$
Тоді для пошуку $c,s$ можна використати наступну систему:
$$\begin{cases}
    cs(a_{jj}-a_{ii})+(c^2-s^2)a_{ij} = 0\\
    c^2 + s^2 = 1
\end{cases}$$

$$
\begin{cases}
    c\sqrt{1-c^2}(a_{jj}-a_{ii})+(2c^2-1)a_{ij} = 0\\
    c^2 + s^2 = 1
\end{cases}$$

<!-- $$c\sqrt{1-c^2}(a_{jj}-a_{ii})=(1-2c^2)a_{ij}$$ 

$$\frac{c\sqrt{1-c^2}}{1-2c^2} = \frac{a_{ij}}{a_{jj}-a_{ii}}$$

$$\frac{c^2(1-c^2)}{1-4c^2+4c^4} = \frac{a^2_{ij}}{(a_{jj}-a_{ii})^2}$$ -->

$$\begin{cases}
c =& \sqrt{\frac{1}{2}\left(1 + (1+\mu^2)^{-\frac{1}{2}}\right)}\\
s =& (\operatorname{sign} \mu) \sqrt{\frac{1}{2}\left(1 - (1+\mu^2)^{-\frac{1}{2}}\right)}\\
\mu =& \displaystyle \frac{2a_{ij}}{a_{ii}-a_{jj}}
\end{cases}$$

In [4]:
def GetCS(A, i, j):
    mu = 2*A[i,j] / (A[i,i] - A[j,j])
    mu1 = np.power(1+mu*mu,-1/2)
    c = np.sqrt((1+mu1)/2)
    s = np.sign(mu) * np.sqrt((1-mu1)/2)
    
    return c,s

def GetT(_A, i, j, c, s):
    T = np.eye(_A.shape[0])
    T[i,i] = c
    T[j,j] = c
    T[i,j] = -s
    T[j,i] = s
    return T

def ApplyT_inplace(_A, i, j, c, s):
    Bi = c*_A[:,i] + s*_A[:,j]
    Bj = -s*_A[:,i] + c*_A[:,j]
    _A[:,i] = Bi
    _A[:,j] = Bj
    Ci = c*_A[i] + s*_A[j]
    Cj = -s*_A[i] + c*_A[j]
    _A[i] = Ci
    _A[j] = Cj
    
def ApplyT(_A, i, j, c, s):
    B = _A.copy()
    Bi = c*_A[:,i] + s*_A[:,j]
    Bj = -s*_A[:,i] + c*_A[:,j]
    B[:,i] = Bi
    B[:,j] = Bj
    Ci = c*B[i] + s*B[j]
    Cj = -s*B[i] + c*B[j]
    B[i] = Ci
    B[j] = Cj
    return B

def ApplyHalfT_inplace(_A, i, j, c, s):
    Bi = c*_A[:,i] + s*_A[:,j]
    Bj = -s*_A[:,i] + c*_A[:,j]
    _A[:,i] = Bi
    _A[:,j] = Bj

In [5]:
def FindPivot(A):
#     I = (A*A).sum(axis=1).argmax()
    
#     J,M = None, -np.inf
#     for j in range(A.shape[0]):
#         if I==j:
#             continue
#         a = np.abs(A[I,j])
#         if a > M:
#             M = a
#             J = j
    I, J, M = None, None, -np.inf
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if i==j:
                continue
            a = np.abs(A[i,j])
            
            if a > M:
                I,J,M = i,j,a

    return I,J

In [21]:
def Run(A, timeout=50, eps=10e-5):
    n = A.shape[0]
    B = A.copy()
    
    Sd = S_d(B)
    S = S_A(B)
    Snd = S-Sd
    T = np.eye(n).astype(float)
    
    print(f"S = {S}, Snd = {Snd}.")
    print(B)
    
    steps = 1
    while True:
        i,j = FindPivot(B)
        c,s = GetCS(B, i, j)
        Tij = GetT(B, i, j, c, s)
        
        ApplyT_inplace(B, i, j, c, s)
        ApplyHalfT_inplace(T, i, j, c, s)
        
        Sd = S_d(B)
        S = S_A(B)
        Snd = S-Sd
        
        print()
        print(f"iteration {steps}. i={i}, j={j}.")
        print("T_ij:\n", Tij)
        print("A:\n", B)
        print(f"S = {S}, Snd = {Snd}.")
        
        if Snd < eps:
            print("Stop. Snd < eps.")
            break
        
        if steps > timeout:
            print("Stop. steps > timeout.")
            break
            
        steps += 1
    
    eigvals = B.diagonal()
    sorting = np.flip(np.argsort(eigvals))
    return eigvals[sorting], T[:,sorting]
    

In [17]:
eigvals, eigvecs = Run(A)

S = 206.411, Snd = 17.6318.
[[7.   0.88 0.93 1.21]
 [0.88 4.16 1.3  0.15]
 [0.93 1.3  6.44 2.  ]
 [1.21 0.15 2.   9.  ]]

iteration 1. i=2, j=3.
T_ij:
 [[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.87722679  0.48007619]
 [ 0.          0.         -0.48007619  0.87722679]]
A:
 [[ 7.          0.88        0.23492872  1.50791528]
 [ 0.88        4.16        1.0683834   0.75568307]
 [ 0.23492872  1.0683834   5.34546847  0.        ]
 [ 1.50791528  0.75568307  0.         10.09453153]]
S = 206.411, Snd = 9.631799999999998.

iteration 2. i=0, j=3.
T_ij:
 [[ 0.92632469  0.          0.          0.37672612]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [-0.37672612  0.          0.          0.92632469]]
A:
 [[ 6.38674735e+00  5.30480175e-01  2.17620277e-01  4.44089210e-16]
 [ 5.30480175e-01  4.16000000e+00  1.06838340e+00  1.03152687e+00]
 [ 2.17620277e-01  1.0

In [24]:
print("Власні значення:", eigvals)
print("\nВласні вектори:")
print(eigvecs)

Власні значення: [10.88007     6.67397864  5.65841894  3.38753242]

Власні вектори:
[[ 0.39098303  0.76786405 -0.48260906 -0.15686161]
 [ 0.16052843  0.31058637  0.34052518  0.87281689]
 [ 0.47622034  0.14232681  0.75262779 -0.43186651]
 [ 0.77108825 -0.54190772 -0.29100259  0.16454915]]


In [23]:
true_eigvals, true_eigvecs = np.linalg.eig(A)
sorting = np.flip(np.argsort(true_eigvals))
print("Еталонні власні значення:", true_eigvals[sorting])
print("\nЕталонні власні вектори:")
print(true_eigvecs[:,sorting])

Еталонні власні значення: [10.88007001  6.67397938  5.65841896  3.38753165]

Еталонні власні вектори:
[[ 0.39101287 -0.76792375 -0.48259516  0.1565374 ]
 [ 0.16054084 -0.31016169  0.3404457  -0.87299661]
 [ 0.47622641 -0.14251517  0.75266671  0.43172983]
 [ 0.77106678  0.54201684 -0.29101797 -0.16426292]]


Бачимо, що власні числа та відповідні їм вектори дуже схожі!

Різниця через точність обчислень та знак